In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [2]:
# Load the dataset
df = pd.read_excel(r'D:\pythonCode\Sector Mapping\Sector_data_with_outliers.xlsx')
print (df)
df = df[df['Outlier'] == 'Inlier']
print(df)
max_tokens = 10
# Split the 'HSN' column into a list of tokens
##df['HSN'] = df['HSN'].str.split(',', n=max_tokens - 1)

                 GSTIN            HSN          Sector  Sector Code  \
0      22BMWPM3936L1ZF       00440013     Advertising            1   
1      22AKZPR2078L1ZD       00440013     Advertising            1   
2      22AUTPB4627K1ZI       00440013     Advertising            1   
3      22ATSPG2600F2Z5       00440013     Advertising            1   
4      22AKFPS5510C1ZO       00440013     Advertising            1   
...                ...            ...             ...          ...   
44380  22ABKPK5356J2ZI         999621  Works Contract           59   
44381  22ARNPR6016R1Z5  999633,995428  Works Contract           59   
44382  22AAFFO7423E1ZM  999691,999699  Works Contract           59   
44383  22AYQPG4386D1ZF         999799  Works Contract           59   
44384  22DKIPK2459A1ZK         999799  Works Contract           59   

       Anomaly_Score Outlier  
0           0.012104  Inlier  
1           0.012104  Inlier  
2           0.012104  Inlier  
3           0.012104  Inlier  
4   

In [4]:
# Text Processing
df['HSN'] = df['HSN'].str.replace(',', ' ')

# Split the data into training and testing sets
X = df['HSN']  # Input feature
y = df['Sector']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:

# Create a CountVectorizer to convert text to numerical features
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)



In [10]:
# Feature Selection using Chi-Squared test
# Select the top K most informative features
k = 7174  # Adjust the number of features (K) as needed
selector = SelectKBest(chi2, k=k)
X_train_selected = selector.fit_transform(X_train_vectorized, y_train)
X_test_selected = selector.transform(X_test_vectorized)

# Train a model on the selected features
model = MultinomialNB()
model.fit(X_train_selected, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_selected)



In [11]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.49


In [12]:
new_data_df = pd.read_excel(r'D:\pythonCode\Sector Mapping\DealerUnmapped.xlsx')

# Preprocess the new data (assuming 'HSN' is the column with tags)
new_data_df['HSN'] = new_data_df['HSN'].str.replace(',', ' ')

# Vectorize the new data
new_data_vectorized = vectorizer.transform(new_data_df['HSN'])

# Make predictions using the trained model
new_predictions = model.predict(new_data_vectorized)

In [13]:
# Add the predictions to the new data DataFrame
new_data_df['Predicted_Type'] = new_predictions

# Save the output to a new Excel file
new_data_df.to_excel(r'D:\pythonCode\Sector Mapping\NaiveBayes.xlsx', index=False)